In [7]:
import engine as ng
from pathlib import Path
import pandas as pd
import json
from typing import List, Callable, Dict, Tuple
import networkx as nx
from matplotlib import pyplot as plt
from engine.graph_io import import_graph
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

data_path = Path("data")

def get_recipe_tf_idf_vectors(json_graph: Dict) -> Tuple[Dict[str, List[int]], np.matrix, TfidfVectorizer]:
    """
    Given a graph in json format, returns a dictionary with the vector representation for all recipes.
    """
    def extract_recipe_info(recipe: Dict) -> List[str]:
        return [x["nombre"] for x in recipe["ingredientes"]]
    
    estimator = TfidfVectorizer()
    recipes = [" ".join(extract_recipe_info(recipe)) for _, recipe in json_graph.items()]
    tfidf = estimator.fit_transform(recipes)
    return {x: tfidf[i] for i, x in enumerate(json_graph)}, tfidf, estimator

def get_vector_representation(estimator: TfidfVectorizer, query: str):
    return estimator.transform([query])


In [3]:
# json_path = data_path / "graphs" / "foodcom" / "food.com.recipe_normalized.json"
json_path = data_path / "graphs" / "recipenlg" / "recipenlg.recipe.json"
recipe_json = json.loads(json_path.read_text())

In [8]:
tfidf_dict, tfidf, vectorizer = get_recipe_tf_idf_vectors(recipe_json)

In [5]:
tfidf_dict[next(iter(tfidf_dict))]

<1x27266 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [6]:
tfidf.shape

(1312871, 27266)

In [15]:
v = get_vector_representation(vectorizer, "white sugar, pineapple and rum")

[0. 0. 0. ... 0. 0. 0.]


In [18]:
from engine.ranking import tfidf_ranking

rank = tfidf_ranking(vectorizer, tfidf, list(tfidf_dict), "white sugar, lemon and rum", 10)

In [19]:
for x in rank:
    print(x)

Fancy Holiday Cookies
Caipirinha Recipe
Strawberry Campari Sorbet
Broad Bean And Parmesan Dip
Grandma Merritt'S Famous Lemon Pound Cake
Roasted Sesame Crackers Made With Thin Toasted Barley Flour
Milk Punch
Rattlesnake Annie'S Hot Pepper Jelly
Steak Or Chicken Marinade
Toasty Macaroons
